In [6]:
# Importing necessary libraries
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [2]:
# Function to apply CLAHE to an X-ray image
def apply_clahe(image):
    # Check if image is already in grayscale
    if len(image.shape) == 3:  # If the image has 3 channels (RGB)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)  # Convert to grayscale

    # Ensure the image is in uint8 format
    if image.dtype != 'uint8':
        image = (image * 255).clip(0, 255).astype('uint8')  # Convert float image to uint8 if necessary

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    enhanced_image = clahe.apply(image)  # Apply CLAHE to the grayscale image
    return enhanced_image

# #Function to preprocess images with CLAHE
# def preprocess_image(image):
#     if isinstance(image, np.ndarray):
#         enhanced_image = apply_clahe(image)
#         enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB)  # Convert back to RGB
#         return enhanced_image
#     return image  # If the input is not an ndarray, return it as is

# Function to preprocess images with CLAHE
def preprocess_image(image):
    if isinstance(image, np.ndarray):
        enhanced_image = apply_clahe(image)
        enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2RGB)  # Convert back to RGB
        # Ensure the final image is float32 for compatibility with ImageDataGenerator
        return enhanced_image.astype('float32') / 255.0  # Scale to [0, 1]
    return image  # If the input is not an ndarray, return it as is


# Function to create ImageDataGenerator with preprocessing and optional validation split
def create_data_generator(preprocess_func, validation_split=None):
    return ImageDataGenerator(
        preprocessing_function=preprocess_func,
        rescale=1./255,  # Scale the pixel values to [0, 1]
        validation_split=validation_split  # Split for validation
    )

# Function to build and compile the model
def build_model():
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification output
    model = Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Training and evaluation function
def train_and_evaluate_model(preprocess_func, model_name):
    print(f"Training model with preprocessing: {model_name}")

    train_datagen = create_data_generator(preprocess_func, validation_split=0.2)  # Split 20% for validation
    test_datagen = create_data_generator(None)

    # Training generator with validation split
    train_generator = train_datagen.flow_from_directory(
        train_dataset_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='training'  # Use the training subset
    )

    val_generator = train_datagen.flow_from_directory(
        train_dataset_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='validation'  # Use the validation subset
    )

    test_generator = test_datagen.flow_from_directory(
        test_dataset_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
    )

    model = build_model()

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // val_generator.batch_size,
        epochs=10  # Set the desired number of epochs
    )

    model.save(f'/content/drive/MyDrive/densenet201_{model_name.lower().replace(" ", "_")}_classifier.h5')

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
    print(f"Test accuracy for {model_name}: {test_acc}")
    return history, test_acc


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# train_dataset_path = '/content/drive/MyDrive/PCD/archive/chest_xray/chest_xray/train'
# test_dataset_path = '/content/drive/MyDrive/PCD/archive/chest_xray/chest_xray/test'

# Define dataset paths
train_dataset_path = '/content/drive/MyDrive/PCD/archive/chest_xray/train'
test_dataset_path = '/content/drive/MyDrive/PCD/archive/chest_xray/test'
val_dataset_path = '/content/drive/MyDrive/PCD/archive/chest_xray/val'

In [7]:
# Train and evaluate model with CLAHE
history_with_clahe, test_acc_with_clahe = train_and_evaluate_model(preprocess_image, "With CLAHE")



Training model with preprocessing: With CLAHE
Found 4180 images belonging to 2 classes.
Found 1044 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


130/130 ━━━━━━━━━━━━━━━━━━━━ 556s 4s/step - accuracy: 0.7182 - loss: 0.5973 - val_accuracy: 0.7432 - val_loss: 0.5697
Epoch 2/10
  1/130 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - accuracy: 0.6875 - loss: 0.6156

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


130/130 ━━━━━━━━━━━━━━━━━━━━ 12s 91ms/step - accuracy: 0.6875 - loss: 0.6156 - val_accuracy: 0.7000 - val_loss: 0.6056
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 92s 557ms/step - accuracy: 0.7426 - loss: 0.5661 - val_accuracy: 0.7432 - val_loss: 0.5629
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7812 - loss: 0.5260 - val_accuracy: 0.7000 - val_loss: 0.6038
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 81s 552ms/step - accuracy: 0.7419 - loss: 0.5623 - val_accuracy: 0.7432 - val_loss: 0.5583
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7188 - loss: 0.5799 - val_accuracy: 0.7000 - val_loss: 0.5991
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 76s 568ms/step - accuracy: 0.7386 - loss: 0.5609 - val_accuracy: 0.7422 - val_loss: 0.5555
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6875 - loss: 0.6271 - val_accuracy: 0.7500 - val_loss: 0.5518
Epoch 9/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 80s 558ms/step - accuracy: 0.7317 - loss: 0.5634 - val_accuracy:

19/19 ━━━━━━━━━━━━━━━━━━━━ 181s 10s/step - accuracy: 0.4355 - loss: 0.8323
Test accuracy for With CLAHE: 0.4440789520740509


In [8]:
# Train and evaluate model without CLAHE
history_without_clahe, test_acc_without_clahe = train_and_evaluate_model(None, "No CLAHE")

Training model with preprocessing: No CLAHE
Found 4180 images belonging to 2 classes.
Found 1044 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 125s 694ms/step - accuracy: 0.8336 - loss: 0.3777 - val_accuracy: 0.9561 - val_loss: 0.1589
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 1.0000 - loss: 0.1089 - val_accuracy: 1.0000 - val_loss: 0.0990
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 70s 514ms/step - accuracy: 0.9371 - loss: 0.1625 - val_accuracy: 0.9629 - val_loss: 0.1227
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0474 - val_accuracy: 1.0000 - val_loss: 0.0706
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 82s 519ms/step - accuracy: 0.9543 - loss: 0.1267 - val_accuracy: 0.9648 - val_loss: 0.1044
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - accuracy: 0.9688 - loss: 0.0919 - val_accuracy: 1.0000 - val_loss: 0.0571
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 80s 508ms/

19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 436ms/step - accuracy: 0.7766 - loss: 0.5414
Test accuracy for No CLAHE: 0.7680920958518982
